In [ ]:
%pip install transformers gradio sympy networkx

In [ ]:
from transformers import pipeline
import networkx as nx
import gradio as gr
import sympy as sp
from collections import defaultdict

In [ ]:
# parse text-based network
def parse_network(input_string):
    edges = []
    reversible_edges = []

    for part in input_string.split(','):
        part = part.strip()
        if '<->' in part:
            a, b = part.split('<->')
            reversible_edges.append((a.strip(), b.strip()))
        elif '->' in part:
            a, b = part.split('->')
            edges.append((a.strip(), b.strip()))

    return edges, reversible_edges

In [ ]:
# build graph using networkx
def build_graph(edges, reversible_edges):
  G = nx.DiGraph()

  G.add_edges_from(edges)
  for a, b in reversible_edges:
    G.add_edge(a, b)
    G.add_edge(b, a)
  return G

In [ ]:
# analyze graph
def analyze_graph(G):
  num_nodes = G.number_of_nodes()
  num_edges = G.number_of_edges()
  is_cyclic = not nx.is_directed_acyclic_graph(G)

  return {"nodes": list(G.nodes),
          "edges": list(G.edges),
          "num_nodes": num_nodes,
          "num_edges":num_edges,
          "is_cyclic": is_cyclic
          }

In [ ]:
%pip install tensorflow

In [ ]:
# Now, import the pipeline
qa = pipeline("text2text-generation", model="google/flan-t5-base")

In [ ]:
# --- Mass-action ODE Generator (rule-based) ---
def mass_action_odes(edges, reversible_edges):
    species = set()
    odes = defaultdict(lambda: 0)
    rate_counter = 1  # Track unique rate constants
    
    # Process irreversible edges (A -> B)
    for a, b in edges:
        A, B = sp.symbols(f"{a} {b}")
        species.update([A, B])
        k = sp.symbols(f'k{rate_counter}')
        rate_counter += 1
        
        odes[A] -= k * A
        odes[B] += k * A

    # Process reversible edges (B <-> C)
    for a, b in reversible_edges:
        A, B = sp.symbols(f"{a} {b}")
        species.update([A, B])
        k_forward = sp.symbols(f'k{rate_counter}')
        rate_counter += 1
        k_reverse = sp.symbols(f'k{rate_counter}')
        rate_counter += 1
        
        # Forward reaction (A -> B)
        odes[A] -= k_forward * A
        odes[B] += k_forward * A
        
        # Reverse reaction (B -> A)
        odes[B] -= k_reverse * B
        odes[A] += k_reverse * B

    return dict(odes)

def format_odes(odes):
    return "\n".join([f"d{var}/dt = {sp.simplify(expr)}" for var, expr in odes.items()])

In [ ]:
def compute_jacobian(odes):
    variables = list(odes.keys())
    F = sp.Matrix([odes[var] for var in variables])
    J = F.jacobian(variables)
    return sp.pretty(J)

In [ ]:
# Define the function that gets responses based on user input
def process_network(input_string, query):
    edges, reversible_edges = parse_network(input_string)
    G = build_graph(edges, reversible_edges)
    info = analyze_graph(G)

    if 'ode' in query.lower():
        ode_sys = mass_action_odes(edges, reversible_edges)
        return format_odes(ode_sys)

    elif 'jacobian' in query.lower():
        ode_sys = mass_action_odes(edges, reversible_edges)
        return f"Jacobian Matrix:\n{compute_jacobian(ode_sys)}"

    elif 'variables' in query.lower():
        return f"There are {info['num_nodes']} variables: {info['nodes']}"

    elif 'edges' in query.lower():
        return f"Edges: {info['edges']}"

    elif 'cyclic' in query.lower():
        return f"Cyclic? {'Yes' if info['is_cyclic'] else 'No'}"

    else:
        # return "Query not understood. Try asking about variables, edges, ODEs, or Jacobian."
        # Use the LLM for other specific queries
        prompt = f"Given the network with nodes: {info['nodes']} and edges: {info['edges']}, answer the query: {query}"
        answer = qa(prompt, max_length=128)[0]['generated_text']

    return answer

# Create the Gradio interface
iface = gr.Interface(
    fn=process_network,
    inputs=["text","text"],  # Network description and user query
    outputs="text",  # Answer text
    # live=True,
    title="Biological Network Query System",
    description="Input a network in the form 'A->B, B<->C' and ask questions like 'How many variables?', 'What are the edges?', or 'Is it cyclic?'"
)

# Launch the interface
iface.launch(share=True)
